In [15]:
from typing import List, Tuple
import os
import skimage
import numpy as np
from sklearn.naive_bayes import GaussianNB
from skimage.color import convert_colorspace
from enum import Enum
import re

import sys  
sys.path.insert(0, '../')
from constants import BOW_TRAIN_PATH, BOW_TEST_PATH


# Helper Methods

In [13]:
class DataClass(Enum):
    NORMAL = 0
    SMOKE = 1
    FIRE = 2

In [23]:
def data_class(path: str) -> DataClass:
    regex = re.compile("([a-zA-Z_]+)([0-9]+)")
    match = regex.match(path)
    if not match:
        raise Exception("Invalid path: " + path)

    data_class = match.group(1)
    if data_class == "fire":
        return DataClass.FIRE
    elif data_class == "smoke":
        return DataClass.SMOKE

    return DataClass.NORMAL


def prepare_features(path: str) -> Tuple[np.ndarray, np.ndarray]:
    images = os.listdir(path)

    X = []
    y = []

    for image_path in images:
        image = skimage.io.imread(path + image_path)
        height, width, _ = image.shape

        # Convert to YCbCr
        image_ycbcr = convert_colorspace(image, "RGB", "YCbCr")

        # Prepare pixel values
        for i in range(height):
            for j in range(width):
                X.append(image_ycbcr[i, j, :])
                y.append(data_class(image_path).value)
    return np.array(X), np.array(y)


def train(path: str) -> GaussianNB:
    model = GaussianNB()
    X, y = prepare_features(path)
    model.fit(X, y)
    return model


In [62]:
def generate_mask(model: GaussianNB, image: np.ndarray) -> np.ndarray:
    height, width, _ = image.shape
    image_ycbcr = convert_colorspace(image, "RGB", "YCbCr")
    prediction = np.array(model.predict(image_ycbcr.reshape(-1, 3)))
    mask = prediction.reshape(height, width)
    mask = np.array(mask/2, dtype=int)
    mask = np.array([mask, mask, mask])
    mask = np.swapaxes(mask, 0, 2)
    mask = np.swapaxes(mask, 0, 1)
    return mask

def test(model: GaussianNB, path: str, threshold: float = 0.15) -> int:
    images = os.listdir(path)
    correct = 0

    for image_path in images:
        image = skimage.io.imread(path + image_path)
        height, width, _ = image.shape

        mask = generate_mask(model, image)
        output_image = np.array(image)
        output_image[mask == 0] = 0
        
        img_out_name = image_path[:-4] + '_out' + image_path[-4:] 
        img_save_path = os.path.join('/Users/tanmayk/Study/Cranfield - AI/IRP/aGANi/data/processed/colorspace', img_out_name)
        skimage.io.imsave(img_save_path, output_image.astype(np.uint8))

        is_fire = mask.sum() /height / width > threshold
        actual = data_class(image_path)
        
        if actual == DataClass.FIRE and is_fire:
            correct += 1
        elif (actual == DataClass.SMOKE or actual == DataClass.NORMAL) and not is_fire:
            correct += 1

    return correct / len(images)

    

# Train

In [24]:
model = train(BOW_TRAIN_PATH)

# Test

In [63]:
accuracy = test(model, BOW_TEST_PATH)
print('accuracy: {:.2f}'.format(accuracy))

/var/folders/dy/7cnns26d2j52cnsqr1ms5fpm0000gn/T/ipykernel_86683/2981107611.py:26: UserWarning: /Users/tanmayk/Study/Cranfield - AI/IRP/aGANi/data/processed/colorspace/not_fire026_out.png is a low contrast image
  skimage.io.imsave(img_save_path, output_image.astype(np.uint8))
/var/folders/dy/7cnns26d2j52cnsqr1ms5fpm0000gn/T/ipykernel_86683/2981107611.py:26: UserWarning: /Users/tanmayk/Study/Cranfield - AI/IRP/aGANi/data/processed/colorspace/not_fire024_out.png is a low contrast image
  skimage.io.imsave(img_save_path, output_image.astype(np.uint8))
/var/folders/dy/7cnns26d2j52cnsqr1ms5fpm0000gn/T/ipykernel_86683/2981107611.py:26: UserWarning: /Users/tanmayk/Study/Cranfield - AI/IRP/aGANi/data/processed/colorspace/not_fire019_out.png is a low contrast image
  skimage.io.imsave(img_save_path, output_image.astype(np.uint8))
/var/folders/dy/7cnns26d2j52cnsqr1ms5fpm0000gn/T/ipykernel_86683/2981107611.py:26: UserWarning: /Users/tanmayk/Study/Cranfield - AI/IRP/aGANi/data/processed/colorspac

accuracy: 0.54
